## Создаем систему мониторинга funding rates (ставок финансирования) на Bybit

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import numpy as np
from matplotlib.animation import FuncAnimation
import warnings
warnings.filterwarnings('ignore')

class BybitFundingMonitor:
    def __init__(self):
        self.base_url = "https://api.bybit.com"
        self.top_symbols = []
        self.funding_data = {}
        
        # Настройка стиля графиков
        plt.style.use('dark_background')
        sns.set_palette("husl")
        
    def get_top_symbols(self, limit=30):
        """Получаем топ символы по объему торгов"""
        try:
            url = f"{self.base_url}/v5/market/tickers"
            params = {"category": "linear"}
            
            response = requests.get(url, params=params, timeout=10)
            data = response.json()
            
            if data['retCode'] == 0:
                tickers = data['result']['list']
                # Фильтруем только USDT пары и сортируем по объему
                usdt_pairs = [
                    ticker for ticker in tickers 
                    if ticker['symbol'].endswith('USDT') and 
                    float(ticker.get('volume24h', 0)) > 0
                ]
                
                # Сортируем по объему торгов
                usdt_pairs.sort(key=lambda x: float(x.get('volume24h', 0)), reverse=True)
                
                self.top_symbols = [ticker['symbol'] for ticker in usdt_pairs[:limit]]
                print(f"✅ Получено {len(self.top_symbols)} топ символов")
                return True
            else:
                print(f"❌ Ошибка получения символов: {data}")
                return False
                
        except Exception as e:
            print(f"❌ Ошибка при получении символов: {e}")
            return False
    
    def get_funding_rate(self, symbol):
        """Получаем текущий funding rate для символа"""
        try:
            url = f"{self.base_url}/v5/market/funding/history"
            params = {
                "category": "linear",
                "symbol": symbol,
                "limit": 1
            }
            
            response = requests.get(url, params=params, timeout=5)
            data = response.json()
            
            if data['retCode'] == 0 and data['result']['list']:
                funding_rate = float(data['result']['list'][0]['fundingRate'])
                return funding_rate
            else:
                return None
                
        except Exception as e:
            print(f"❌ Ошибка получения funding rate для {symbol}: {e}")
            return None
    
    def get_predicted_funding_rate(self, symbol):
        """Получаем предсказанный funding rate"""
        try:
            url = f"{self.base_url}/v5/market/tickers"
            params = {
                "category": "linear",
                "symbol": symbol
            }
            
            response = requests.get(url, params=params, timeout=5)
            data = response.json()
            
            if data['retCode'] == 0 and data['result']['list']:
                ticker = data['result']['list'][0]
                predicted_rate = ticker.get('nextFundingTime', None)
                funding_rate = ticker.get('fundingRate', None)
                
                if funding_rate:
                    return float(funding_rate)
                else:
                    return None
            else:
                return None
                
        except Exception as e:
            return None
    
    def calculate_annual_yield(self, funding_rate):
        """Рассчитываем годовую доходность"""
        if funding_rate is None:
            return 0
        
        # Funding происходит каждые 8 часов (3 раза в день)
        # В году 365 дней = 365 * 3 = 1095 funding событий
        daily_rate = funding_rate * 3  # 3 раза в день
        annual_rate = daily_rate * 365  # на год
        
        return annual_rate * 100  # в процентах
    
    def collect_funding_data(self):
        """Собираем данные по всем символам"""
        print(f"🔄 Обновление данных в {datetime.now().strftime('%H:%M:%S')}")
        
        current_data = []
        
        for symbol in self.top_symbols:
            funding_rate = self.get_predicted_funding_rate(symbol)
            
            if funding_rate is not None:
                annual_yield = self.calculate_annual_yield(funding_rate)
                
                current_data.append({
                    'symbol': symbol,
                    'funding_rate': funding_rate,
                    'funding_rate_pct': funding_rate * 100,
                    'annual_yield': annual_yield,
                    'timestamp': datetime.now()
                })
        
        # Обновляем данные
        df = pd.DataFrame(current_data)
        if not df.empty:
            df = df.sort_values('annual_yield', ascending=False)
            self.funding_data = df
            
        return df
    
    def create_dashboard(self):
        """Создаем дашборд с визуализацией"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 12))
        fig.suptitle('📊 BYBIT FUNDING RATES MONITOR - REAL TIME', 
                     fontsize=20, fontweight='bold', color='white')
        
        def update_plots(frame):
            # Очищаем все графики
            ax1.clear()
            ax2.clear()
            ax3.clear()
            ax4.clear()
            
            # Собираем новые данные
            df = self.collect_funding_data()
            
            if df.empty:
                return
            
            # График 1: Топ 15 по годовой доходности
            top_15 = df.head(15)
            colors = ['green' if x > 0 else 'red' for x in top_15['annual_yield']]
            
            bars1 = ax1.barh(range(len(top_15)), top_15['annual_yield'], color=colors, alpha=0.8)
            ax1.set_yticks(range(len(top_15)))
            ax1.set_yticklabels([s.replace('USDT', '') for s in top_15['symbol']], fontsize=10)
            ax1.set_xlabel('Годовая доходность (%)', fontsize=12, fontweight='bold')
            ax1.set_title('🚀 ТОП-15 ПО ДОХОДНОСТИ', fontsize=14, fontweight='bold', color='lime')
            ax1.grid(True, alpha=0.3)
            ax1.invert_yaxis()
            
            # Добавляем значения на бары
            for i, (bar, value) in enumerate(zip(bars1, top_15['annual_yield'])):
                ax1.text(value + (max(top_15['annual_yield']) * 0.01), i, 
                        f'{value:.1f}%', va='center', fontweight='bold')
            
            # График 2: Текущие funding rates (%)
            current_rates = df.head(15)
            colors2 = ['green' if x > 0 else 'red' for x in current_rates['funding_rate_pct']]
            
            bars2 = ax2.barh(range(len(current_rates)), current_rates['funding_rate_pct'], 
                           color=colors2, alpha=0.8)
            ax2.set_yticks(range(len(current_rates)))
            ax2.set_yticklabels([s.replace('USDT', '') for s in current_rates['symbol']], fontsize=10)
            ax2.set_xlabel('Текущий Funding Rate (%)', fontsize=12, fontweight='bold')
            ax2.set_title('📈 ТЕКУЩИЕ FUNDING RATES', fontsize=14, fontweight='bold', color='cyan')
            ax2.grid(True, alpha=0.3)
            ax2.invert_yaxis()
            
            # Добавляем значения
            for i, (bar, value) in enumerate(zip(bars2, current_rates['funding_rate_pct'])):
                ax2.text(value + (max(abs(current_rates['funding_rate_pct'])) * 0.01), i, 
                        f'{value:.4f}%', va='center', fontweight='bold', fontsize=8)
            
            # График 3: Распределение доходности
            ax3.hist(df['annual_yield'], bins=20, color='orange', alpha=0.7, edgecolor='black')
            ax3.set_xlabel('Годовая доходность (%)', fontsize=12, fontweight='bold')
            ax3.set_ylabel('Количество пар', fontsize=12, fontweight='bold')
            ax3.set_title('📊 РАСПРЕДЕЛЕНИЕ ДОХОДНОСТИ', fontsize=14, fontweight='bold', color='orange')
            ax3.grid(True, alpha=0.3)
            ax3.axvline(x=0, color='white', linestyle='--', alpha=0.8)
            
            # График 4: Сводная статистика
            ax4.axis('off')
            
            # Статистика
            positive_count = len(df[df['annual_yield'] > 0])
            negative_count = len(df[df['annual_yield'] < 0])
            max_yield = df['annual_yield'].max()
            min_yield = df['annual_yield'].min()
            avg_yield = df['annual_yield'].mean()
            
            best_pair = df.loc[df['annual_yield'].idxmax(), 'symbol'] if not df.empty else 'N/A'
            worst_pair = df.loc[df['annual_yield'].idxmin(), 'symbol'] if not df.empty else 'N/A'
            
            stats_text = f"""
📊 СТАТИСТИКА РЫНКА

🎯 ЛУЧШАЯ ВОЗМОЖНОСТЬ:
   {best_pair.replace('USDT', '')}: {max_yield:.2f}% годовых

📉 ХУДШАЯ ПАРА:
   {worst_pair.replace('USDT', '')}: {min_yield:.2f}% годовых

📈 СРЕДНЯЯ ДОХОДНОСТЬ: {avg_yield:.2f}%

✅ ПОЛОЖИТЕЛЬНЫЕ ПАРЫ: {positive_count}
❌ ОТРИЦАТЕЛЬНЫЕ ПАРЫ: {negative_count}

⏰ ОБНОВЛЕНО: {datetime.now().strftime('%H:%M:%S')}

💡 СОВЕТ: Ищите пары с высоким
   положительным funding rate
   для SHORT позиций
            """
            
            ax4.text(0.05, 0.95, stats_text, transform=ax4.transAxes, 
                    fontsize=12, verticalalignment='top', fontweight='bold',
                    bbox=dict(boxstyle="round,pad=0.5", facecolor="navy", alpha=0.8),
                    color='white')
            
            plt.tight_layout()
            
        # Запускаем анимацию
        ani = FuncAnimation(fig, update_plots, interval=5000, cache_frame_data=False)
        return ani
    
    def run_monitor(self):
        """Запускаем мониторинг"""
        print("🚀 Запуск Bybit Funding Rate Monitor...")
        
        # Получаем топ символы
        if not self.get_top_symbols():
            print("❌ Не удалось получить список символов")
            return
        
        print(f"📈 Мониторинг {len(self.top_symbols)} пар...")
        print("🔄 Обновление каждые 5 секунд")
        print("💡 Закройте окно графика для остановки")
        
        # Создаем и показываем дашборд
        ani = self.create_dashboard()
        plt.show()

# Функция для простого мониторинга в консоли
def console_monitor():
    """Простой мониторинг в консоли"""
    monitor = BybitFundingMonitor()
    
    if not monitor.get_top_symbols(20):
        return
    
    print("\n" + "="*80)
    print("📊 BYBIT FUNDING RATES - CONSOLE MODE")
    print("="*80)
    
    try:
        while True:
            df = monitor.collect_funding_data()
            
            if not df.empty:
                print(f"\n⏰ Обновлено: {datetime.now().strftime('%H:%M:%S')}")
                print("-" * 60)
                
                # Топ 10 по доходности
                top_10 = df.head(10)
                
                for idx, row in top_10.iterrows():
                    symbol = row['symbol'].replace('USDT', '')
                    annual_yield = row['annual_yield']
                    funding_rate = row['funding_rate_pct']
                    
                    status = "🟢" if annual_yield > 0 else "🔴"
                    
                    print(f"{status} {symbol:8} | "
                          f"Funding: {funding_rate:8.4f}% | "
                          f"Годовых: {annual_yield:8.2f}%")
                
                print("\n💡 Для графического интерфейса запустите: monitor.run_monitor()")
            
            time.sleep(5)
            
    except KeyboardInterrupt:
        print("\n\n👋 Мониторинг остановлен")

if __name__ == "__main__":
    # Создаем экземпляр монитора
    monitor = BybitFundingMonitor()
    
    print("Выберите режим:")
    print("1. Графический интерфейс (рекомендуется)")
    print("2. Консольный режим")
    
    choice = input("Введите номер (1 или 2): ").strip()
    
    if choice == "2":
        console_monitor()
    else:
        monitor.run_monitor()

Выберите режим:
1. Графический интерфейс (рекомендуется)
2. Консольный режим


Введите номер (1 или 2):  2


✅ Получено 20 топ символов

📊 BYBIT FUNDING RATES - CONSOLE MODE
🔄 Обновление данных в 13:38:50

⏰ Обновлено: 13:38:58
------------------------------------------------------------
🟢 SHIB1000 | Funding:   0.0100% | Годовых:    10.95%
🟢 GALA     | Funding:   0.0100% | Годовых:    10.95%
🟢 1000PEPE | Funding:   0.0085% | Годовых:     9.27%
🟢 HMSTR    | Funding:   0.0050% | Годовых:     5.48%
🟢 DEGEN    | Funding:   0.0050% | Годовых:     5.48%
🟢 1000000BABYDOGE | Funding:   0.0050% | Годовых:     5.48%
🟢 DOGS     | Funding:   0.0050% | Годовых:     5.48%
🟢 NOT      | Funding:   0.0050% | Годовых:     5.48%
🟢 MEW      | Funding:   0.0050% | Годовых:     5.48%
🟢 BOME     | Funding:   0.0050% | Годовых:     5.48%

💡 Для графического интерфейса запустите: monitor.run_monitor()
🔄 Обновление данных в 13:39:03

⏰ Обновлено: 13:39:10
------------------------------------------------------------
🟢 SHIB1000 | Funding:   0.0100% | Годовых:    10.95%
🟢 GALA     | Funding:   0.0100% | Годовых:    10.95